# Requisitos previos
<hr>

In [87]:
# --------------------------- REQUISITOS PREVIOS ---------------------------

# Pandas, openbabel y plip (quitar el # de las siguientes filas en caso de no tener instalados)
#!conda install -q -y -c anaconda pandas
#!conda install -q -y -c conda-forge openbabel
#!conda install -y -q -c conda-forge plip

# Separar ligandos inhibidores
<hr>

In [1]:
import os
import glob
import pandas as pd

# Buscar archivos .pdbqt
pdbqts = glob.glob("docking/inhibidores/*.pdbqt")

# Crear carpeta de salida
carpeta_salida = "Mejores_poses/inhibidores"
os.makedirs(carpeta_salida, exist_ok=True)

# Listas para el DataFrame
nombres = []
proteinas = []
rutas = []
afinidades = []
CNNs = []
CNN_afinidades = []

# Procesar cada archivo
for ruta in pdbqts:
    # Extraer nombre y proteína
    nombre_archivo = os.path.basename(ruta)
    molecula = nombre_archivo.split("_L_")[0]
    target = nombre_archivo.split("_L_")[1].split(".")[0]
    
    # Leer el archivo
    with open(ruta, 'r') as f:
        lines = f.readlines()

    # Extraer solo la primera pose (hasta ENDMDL)
    new_lines = []
    for line in lines:
        new_lines.append(line)
        if line.strip() == "ENDMDL":
            break
    
    # Guardar archivo con solo la primera pose
    ruta_salida = os.path.join(carpeta_salida, f"{molecula}_{target}.pdbqt")
    with open(ruta_salida, 'w') as f_out:
        for line in new_lines:
            f_out.write(line)
    
    # Extraer valores de afinidad y CNN
    try:
        afinidades.append(new_lines[1].split()[2])
        CNNs.append(new_lines[2].split()[2])
        CNN_afinidades.append(new_lines[3].split()[2])
    except IndexError:
        afinidades.append(None)
        CNNs.append(None)
        CNN_afinidades.append(None)

    # Guardar datos para el DataFrame
    nombres.append(molecula)
    proteinas.append(target)
    rutas.append(ruta_salida)  # ← CAMBIO AQUÍ

# Crear DataFrame final
df = pd.DataFrame({
    "Molecula": nombres,
    "Target": proteinas,
    "Ruta": rutas,
    "Afinidad": afinidades,
    "CNN": CNNs,
    "CNN afinity": CNN_afinidades
})

In [2]:
df.isna().sum()

Molecula       0
Target         0
Ruta           0
Afinidad       0
CNN            0
CNN afinity    0
dtype: int64

In [3]:
df

,Molecula,Target,Ruta,Afinidad,CNN,CNN afinity
0,EGFR_P_MAPK1_158,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_158_MAP...,-9.65617466,active,('A'
1,EGFR_P_MAPK1_159,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_159_MAP...,-8.76622868,active,('A'
2,EGFR_P_MAPK1_160,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_160_MAP...,-10.7767897,active,('A'
3,EGFR_P_MAPK1_161,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_161_MAP...,-9.3945322,active,('A'
4,EGFR_P_MAPK1_162,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_162_MAP...,-10.0634918,active,('A'
...,...,...,...,...,...,...
316,SRC_P_STAT3_312,STAT3,Mejores_poses/inhibidores/SRC_P_STAT3_312_STAT...,-5.76559353,active,('A'
317,SRC_P_STAT3_313,STAT3,Mejores_poses/inhibidores/SRC_P_STAT3_313_STAT...,-7.74202442,active,('A'
318,SRC_P_STAT3_314,STAT3,Mejores_poses/inhibidores/SRC_P_STAT3_314_STAT...,-7.38600349,active,('A'
319,SRC_P_STAT3_315,STAT3,Mejores_poses/inhibidores/SRC_P_STAT3_315_STAT...,-8.22408199,active,('A'


In [4]:
df["Target"].value_counts()

Target
EGFR     158
STAT3     86
MAPK1     48
SRC       29
Name: count, dtype: int64

# Determinar multitargets inhibidores
<hr>

In [5]:
import pandas as pd

# 1. Filtrar moléculas multitarget
moleculas_multitarget = df.groupby('Molecula')['Target'].nunique()
moleculas_multitarget = moleculas_multitarget[moleculas_multitarget > 1].index

df_multi = df[df['Molecula'].isin(moleculas_multitarget)].copy()

# 2. Agregar un índice para cada target por molécula
df_multi['Target_n'] = df_multi.groupby('Molecula').cumcount() + 1

# 3. Pivotear todas las columnas relevantes, incluyendo Ruta
pivoted = df_multi.pivot(index='Molecula', columns='Target_n', values=['Target', 'Afinidad', 'CNN', 'CNN afinity', 'Ruta'])

# 4. Aplanar el MultiIndex de columnas
pivoted.columns = [f"{col}_{i}" for col, i in pivoted.columns]

# 5. Restablecer el índice para tener "Molecula" como columna normal
resultado = pivoted.reset_index()

# 6. Reordenar columnas si lo deseas (opcional)
cols = ['Molecula'] + sorted([col for col in resultado.columns if col != 'Molecula'])
resultado = resultado[cols]


In [6]:
resultado

,Molecula,Afinidad_1,Afinidad_2,CNN afinity_1,CNN afinity_2,CNN_1,CNN_2,Ruta_1,Ruta_2,Target_1,Target_2
0,SRC_P_EGFR_STAT3_157,-10.0819273,-8.41119766,('A',('A',active,active,Mejores_poses/inhibidores/SRC_P_EGFR_STAT3_157...,Mejores_poses/inhibidores/SRC_P_EGFR_STAT3_157...,EGFR,STAT3


In [7]:
# 1. Obtener lista de moléculas ya incluidas en resultado
moleculas_a_eliminar = resultado['Molecula'].unique()

# 2. Filtrar df para excluir esas moléculas
df_filtrado = df[~df['Molecula'].isin(moleculas_a_eliminar)].copy()


In [8]:
df_filtrado

,Molecula,Target,Ruta,Afinidad,CNN,CNN afinity
0,EGFR_P_MAPK1_158,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_158_MAP...,-9.65617466,active,('A'
1,EGFR_P_MAPK1_159,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_159_MAP...,-8.76622868,active,('A'
2,EGFR_P_MAPK1_160,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_160_MAP...,-10.7767897,active,('A'
3,EGFR_P_MAPK1_161,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_161_MAP...,-9.3945322,active,('A'
4,EGFR_P_MAPK1_162,MAPK1,Mejores_poses/inhibidores/EGFR_P_MAPK1_162_MAP...,-10.0634918,active,('A'
...,...,...,...,...,...,...
316,SRC_P_STAT3_312,STAT3,Mejores_poses/inhibidores/SRC_P_STAT3_312_STAT...,-5.76559353,active,('A'
317,SRC_P_STAT3_313,STAT3,Mejores_poses/inhibidores/SRC_P_STAT3_313_STAT...,-7.74202442,active,('A'
318,SRC_P_STAT3_314,STAT3,Mejores_poses/inhibidores/SRC_P_STAT3_314_STAT...,-7.38600349,active,('A'
319,SRC_P_STAT3_315,STAT3,Mejores_poses/inhibidores/SRC_P_STAT3_315_STAT...,-8.22408199,active,('A'


# Guardar datos de compuestos
<hr>

In [9]:
df_filtrado.to_csv("Resultados_docking/Inhibidores/Duales.csv")

In [10]:
resultado.to_csv("Resultados_docking/Inhibidores/Triples.csv")

# Separar ligandos de las bases
<hr>

In [11]:
import os
import glob
import pandas as pd

# Buscar archivos .pdbqt
pdbqts = glob.glob("docking/bases_conjuntas/*.pdbqt")

# Crear carpeta de salida
carpeta_salida = "Mejores_poses/bases_conjuntas"
os.makedirs(carpeta_salida, exist_ok=True)

# Listas para el DataFrame
nombres = []
proteinas = []
rutas = []
afinidades = []
CNNs = []
CNN_afinidades = []

# Procesar cada archivo
for ruta in pdbqts:
    # Extraer nombre y proteína
    nombre_archivo = os.path.basename(ruta)
    molecula = nombre_archivo.split("_L_")[0]
    target = nombre_archivo.split("_L_")[1].split(".")[0]
    
    # Leer el archivo
    with open(ruta, 'r') as f:
        lines = f.readlines()

    # Extraer solo la primera pose (hasta ENDMDL)
    new_lines = []
    for line in lines:
        new_lines.append(line)
        if line.strip() == "ENDMDL":
            break
    
    # Guardar archivo con solo la primera pose
    ruta_salida = os.path.join(carpeta_salida, f"{molecula}_{target}.pdbqt")
    with open(ruta_salida, 'w') as f_out:
        for line in new_lines:
            f_out.write(line)
    
    # Extraer valores de afinidad y CNN
    try:
        afinidades.append(new_lines[1].split()[2])
        CNNs.append(new_lines[2].split()[2])
        CNN_afinidades.append(new_lines[3].split()[2])
    except IndexError:
        afinidades.append(None)
        CNNs.append(None)
        CNN_afinidades.append(None)

    # Guardar datos para el DataFrame
    nombres.append(molecula)
    proteinas.append(target)
    rutas.append(ruta_salida)  # ← CAMBIO AQUÍ

# Crear DataFrame final
df = pd.DataFrame({
    "Molecula": nombres,
    "Target": proteinas,
    "Ruta": rutas,
    "Afinidad": afinidades,
    "CNN": CNNs,
    "CNN afinity": CNN_afinidades
})


In [12]:
df

,Molecula,Target,Ruta,Afinidad,CNN,CNN afinity
0,EGFR_MAPK1_0,EGFR,Mejores_poses/bases_conjuntas/EGFR_MAPK1_0_EGF...,-9.92698479,active,('A'
1,EGFR_MAPK1_0,MAPK1,Mejores_poses/bases_conjuntas/EGFR_MAPK1_0_MAP...,-9.6376791,active,('A'
2,EGFR_MAPK1_100,EGFR,Mejores_poses/bases_conjuntas/EGFR_MAPK1_100_E...,-9.75053692,active,('A'
3,EGFR_MAPK1_100,MAPK1,Mejores_poses/bases_conjuntas/EGFR_MAPK1_100_M...,-9.20737648,active,('A'
4,EGFR_MAPK1_101,EGFR,Mejores_poses/bases_conjuntas/EGFR_MAPK1_101_E...,-10.0274315,active,('A'
...,...,...,...,...,...,...
1175,EGFR_STAT3_590,STAT3,Mejores_poses/bases_conjuntas/EGFR_STAT3_590_S...,-6.15254211,active,('A'
1176,EGFR_STAT3_591,EGFR,Mejores_poses/bases_conjuntas/EGFR_STAT3_591_E...,-6.48040104,active,('A'
1177,EGFR_STAT3_591,STAT3,Mejores_poses/bases_conjuntas/EGFR_STAT3_591_S...,-5.40995264,active,('A'
1178,MAPK1_SRC_592,MAPK1,Mejores_poses/bases_conjuntas/MAPK1_SRC_592_MA...,-9.87538052,active,('A'


In [13]:
df["Target"].value_counts()

Target
EGFR     589
MAPK1    555
SRC       20
STAT3     16
Name: count, dtype: int64

In [14]:
import pandas as pd

# 1. Filtrar moléculas multitarget
moleculas_multitarget = df.groupby('Molecula')['Target'].nunique()
moleculas_multitarget = moleculas_multitarget[moleculas_multitarget > 1].index

df_multi = df[df['Molecula'].isin(moleculas_multitarget)].copy()

# 2. Agregar un índice para cada target por molécula
df_multi['Target_n'] = df_multi.groupby('Molecula').cumcount() + 1

# 3. Pivotear todas las columnas relevantes, incluyendo Ruta
pivoted = df_multi.pivot(index='Molecula', columns='Target_n', values=['Target', 'Afinidad', 'CNN', 'CNN afinity', 'Ruta'])

# 4. Aplanar el MultiIndex de columnas
pivoted.columns = [f"{col}_{i}" for col, i in pivoted.columns]

# 5. Restablecer el índice para tener "Molecula" como columna normal
resultado = pivoted.reset_index()

# 6. Reordenar columnas si lo deseas (opcional)
cols = ['Molecula'] + sorted([col for col in resultado.columns if col != 'Molecula'])
resultado = resultado[cols]


In [15]:
resultado

,Molecula,Afinidad_1,Afinidad_2,CNN afinity_1,CNN afinity_2,CNN_1,CNN_2,Ruta_1,Ruta_2,Target_1,Target_2
0,EGFR_MAPK1_0,-9.92698479,-9.6376791,('A',('A',active,active,Mejores_poses/bases_conjuntas/EGFR_MAPK1_0_EGF...,Mejores_poses/bases_conjuntas/EGFR_MAPK1_0_MAP...,EGFR,MAPK1
1,EGFR_MAPK1_1,-10.2506361,-9.38095093,('A',('A',active,active,Mejores_poses/bases_conjuntas/EGFR_MAPK1_1_EGF...,Mejores_poses/bases_conjuntas/EGFR_MAPK1_1_MAP...,EGFR,MAPK1
2,EGFR_MAPK1_10,-9.45638943,-9.22634697,('A',('A',active,active,Mejores_poses/bases_conjuntas/EGFR_MAPK1_10_EG...,Mejores_poses/bases_conjuntas/EGFR_MAPK1_10_MA...,EGFR,MAPK1
3,EGFR_MAPK1_100,-9.75053692,-9.20737648,('A',('A',active,active,Mejores_poses/bases_conjuntas/EGFR_MAPK1_100_E...,Mejores_poses/bases_conjuntas/EGFR_MAPK1_100_M...,EGFR,MAPK1
4,EGFR_MAPK1_101,-10.0274315,-10.5229015,('A',('A',active,active,Mejores_poses/bases_conjuntas/EGFR_MAPK1_101_E...,Mejores_poses/bases_conjuntas/EGFR_MAPK1_101_M...,EGFR,MAPK1
...,...,...,...,...,...,...,...,...,...,...,...
585,EGFR_STAT3_588,-8.90237713,-7.6079812,('A',('A',active,active,Mejores_poses/bases_conjuntas/EGFR_STAT3_588_E...,Mejores_poses/bases_conjuntas/EGFR_STAT3_588_S...,EGFR,STAT3
586,EGFR_STAT3_589,-6.21409321,-5.0345192,('A',('A',active,active,Mejores_poses/bases_conjuntas/EGFR_STAT3_589_E...,Mejores_poses/bases_conjuntas/EGFR_STAT3_589_S...,EGFR,STAT3
587,EGFR_STAT3_590,-6.86469984,-6.15254211,('A',('A',active,active,Mejores_poses/bases_conjuntas/EGFR_STAT3_590_E...,Mejores_poses/bases_conjuntas/EGFR_STAT3_590_S...,EGFR,STAT3
588,EGFR_STAT3_591,-6.48040104,-5.40995264,('A',('A',active,active,Mejores_poses/bases_conjuntas/EGFR_STAT3_591_E...,Mejores_poses/bases_conjuntas/EGFR_STAT3_591_S...,EGFR,STAT3


# Guardar datos de compuestos
<hr>

In [16]:
resultado.to_csv("Resultados_docking/Bases_conjuntas/Duales.csv")